In [1]:
import numpy as np
import math

## Q1

In [2]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

saut_1 = lambda x: np.random.choice([-1, 1], size=x, p=[0.5, 0.5])
saut_2 = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, p=[1/6, 1/3, 1/2, 1/2, 1/3, 1/6])
P0 = 35
T = 4*60*60
lamb = 1/300

### Q1 -1 Trajectoire

In [11]:
from numba import jit, prange

@jit(nopython=True, parallel=True)
def inf_echantillion(N, J, res, P0): # Function is compiled to machine code when called the first time
  for i in prange(len(N) - 1):
    somme = P0
    for j in range(N[i], N[i + 1]):
      somme += J[j]
      if somme < 0:
        res[i] = 1
        break
  return res

def trajectoire(P0, T, lamb, saut, size):
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à N pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] + 1 et N[i + 1] suit la loi voulue
  J = saut(N[-1] + 1)
  res = np.zeros(size, dtype=bool)
  res = inf_echantillion(N, J, res, P0)
  proba = (res).mean()
  return proba

In [13]:
%%time
print(trajectoire(P0, T, lamb, saut_1, size=int(1e7)))

3e-07
Wall time: 12.9 s


In [8]:
%%time
print(trajectoire(P0, T, lamb, saut_1, size=int(1e7)))

1e-07
Wall time: 13.4 s


###  Q1 - 2 Quantile


In [6]:
# Poisson composé naïf
def Poisson_compose(P0, T, lamb, saut, size):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à N pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] + 1 et N[i + 1] suit la loi voulue
  J = saut(N[-1] + 1)               # Sauts
  M = np.add.reduceat(J, N[:-1])    # On veut la somme entre l'indice N[i] + 1 et N[i + 1]
  return P0 + M # échantillon de taille (size)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question1_2_naive(size, quant, saut, P0, T, lamb):
  return quantile(Poisson_compose(P0, T, lamb, saut, size), size, quant)

In [7]:
%%time
size = int(1e6)
quant = 1e-4
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

(9, 61)
Wall time: 1.41 s
